# Predict DL methods
- Environment setup
- Load both 2017, 2018 original and corrected CICIDS datasets
- Apply methods

### Environment setup

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!wget https://downloads.rclone.org/v1.63.0/rclone-v1.63.0-linux-amd64.deb
!apt install ./rclone-v1.63.0-linux-amd64.deb
!rclone config
!sudo apt-get -y install fuse3


--2023-08-19 07:43:18--  https://downloads.rclone.org/v1.63.0/rclone-v1.63.0-linux-amd64.deb
Resolving downloads.rclone.org (downloads.rclone.org)... 95.217.6.16, 2a01:4f9:c012:7154::1
Connecting to downloads.rclone.org (downloads.rclone.org)|95.217.6.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18373062 (18M) [application/vnd.debian.binary-package]
Saving to: ‘rclone-v1.63.0-linux-amd64.deb’

rclone-v1.63.0-linu 100%[===================>]  17.52M  7.16MB/s    in 2.4s    

2023-08-19 07:43:22 (7.16 MB/s) - ‘rclone-v1.63.0-linux-amd64.deb’ saved [18373062/18373062]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'rclone' instead of './rclone-v1.63.0-linux-amd64.deb'
The following NEW packages will be installed:
  rclone
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 0 B/18.4 MB of archives.
After this operation, 56.7 MB of additional disk space will be used.
Get

In [3]:
!sudo mkdir /content/onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

nohup: appending output to 'nohup.out'


### Load both 2017, 2018 original and corrected CICIDS datasets

In [1]:
# Import Necessary Libraries
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt # plotting
import seaborn as sns
from sklearn.manifold import TSNE
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score, accuracy_score

# for Deep Learning
import tensorflow as tf

from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from keras import regularizers
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

import time
import os


In [5]:
pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.8 MB/s eta 0:00:00


In [2]:
# data2017_corrected = pd.read_parquet('/content/onedrive/data_2017_corrected.parquet', engine="fastparquet")
# data2018_original = pd.read_parquet('/content/onedrive/data_2018_original.parquet', engine="fastparquet")
data2018_corrected = pd.read_parquet('/content/onedrive/data_2018_corrected_sampling.parquet', engine="fastparquet")

In [21]:
# data2017_corrected_resampling = pd.read_parquet('/content/onedrive/data_2017_corrected_resampling.parquet', engine="fastparquet")
# data2018_original_resampling = pd.read_parquet('/content/onedrive/data_2018_original_resampling.parquet', engine="fastparquet")
data2018_corrected_resampling = pd.read_parquet('/content/onedrive/data_2018_corrected_sampling_resampling.parquet', engine="fastparquet")

In [7]:
data2017_corrected = data2017_corrected.drop(['Fwd RST Flags',
                                                   'Bwd RST Flags',
                                                   'ICMP Code',
                                                   'ICMP Type',
                                                   'Total TCP Flow Time',
                                                   'Attempted Category'], axis = 1)

In [ ]:
data2017_corrected_resampling = data2017_corrected_resampling.drop(['Fwd RST Flags',
                                                   'Bwd RST Flags',
                                                   'ICMP Code',
                                                   'ICMP Type',
                                                   'Total TCP Flow Time',
                                                   'Attempted Category'], axis = 1)

In [3]:
data2018_corrected = data2018_corrected.drop(['Fwd RST Flags',
                                              'Bwd RST Flags',
                                              'ICMP Code',
                                              'ICMP Type',
                                              'Total TCP Flow Time'], axis = 1)

In [22]:
data2018_corrected_resampling = data2018_corrected_resampling.drop(['Fwd RST Flags',
                                              'Bwd RST Flags',
                                              'ICMP Code',
                                              'ICMP Type',
                                              'Total TCP Flow Time'], axis = 1)

In [4]:
def get_x_y(df):
    X = df.loc[:, df.columns != 'label_encoded']
    y = df['label_encoded']
    return X, y

In [5]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [6]:
# Load the saved model
input_dim = 76 #num of feature columns

class AutoEncoder(Model):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = tf.keras.Sequential([
                  tf.keras.layers.Dense(64, activation="relu"),
                  tf.keras.layers.Dense(32, activation="relu"),
                  tf.keras.layers.Dense(16, activation="relu"),
              ])
        self.decoder = tf.keras.Sequential([
                  tf.keras.layers.Dense(32, activation="relu"),
                  tf.keras.layers.Dense(64, activation="relu"),
                  tf.keras.layers.Dense(input_dim, activation="sigmoid")
              ])
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        label_pred = self.output_layer(decoded)
        return label_pred

autoencoder = AutoEncoder()


In [ ]:
def predict_ae_from_upstream_model(df, evaluate_x, evaluate_y, is_resampling = False):
  autoencoder.compile(loss='mae',
                    optimizer='rmsprop',
                    metrics=['acc',f1_m, precision_m, recall_m])
  if (is_resampling):
    autoencoder.load_weights("/content/onedrive/models/autoencoder_classifier_resampling.ckpt")
  else:
    autoencoder.load_weights("/content/onedrive/models/autoencoder_classifier.ckpt")

  ypred = autoencoder.predict(evaluate_x)
  # Define a threshold for converting predictions to binary class labels
  threshold = 0.5

  # Convert predictions to binary class labels
  predicted_classes = (ypred >= threshold).astype(int)
  df['ypreds_ae'] = predicted_classes
  print(classification_report(evaluate_y, predicted_classes))
  autoencoder.evaluate(evaluate_x, evaluate_y, verbose=2)

### 2017 improved

In [ ]:
X_test, y_test = get_x_y(data2017_corrected)

data_test_2017_corrected = pd.concat([X_test, y_test], axis=1)

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [ ]:
predict_ae_from_upstream_model(data_test_2017_corrected, X_test, y_test)

65625/65625 [==============================] - 76s 1ms/step
              precision    recall  f1-score   support

           0       0.85      0.99      0.91   1594540
           1       0.94      0.44      0.60    505431

    accuracy                           0.86   2099971
   macro avg       0.89      0.71      0.75   2099971
weighted avg       0.87      0.86      0.84   2099971

65625/65625 - 80s - loss: 0.1428 - acc: 0.8572 - f1_m: 0.2182 - precision_m: 0.3467 - recall_m: 0.1860 - 80s/epoch - 1ms/step


### Resampling

In [ ]:
X_test, y_test = get_x_y(data2017_corrected_resampling)

data_test_2017_corrected_r = pd.concat([X_test, y_test], axis=1)

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [ ]:
predict_ae_from_upstream_model(data_test_2017_corrected_r, X_test, y_test, True)

31590/31590 [==============================] - 38s 1ms/step
              precision    recall  f1-score   support

           0       0.77      0.89      0.82    505431
           1       0.87      0.73      0.79    505431

    accuracy                           0.81   1010862
   macro avg       0.82      0.81      0.81   1010862
weighted avg       0.82      0.81      0.81   1010862

31590/31590 - 40s - loss: 0.1891 - acc: 0.8107 - f1_m: 0.3952 - precision_m: 0.4931 - recall_m: 0.3669 - 40s/epoch - 1ms/step


### 2018 original

In [ ]:
X_test, y_test = get_x_y(data2018_original)
del data2018_original
data_test_2018_original = pd.concat([X_test, y_test], axis=1)

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [ ]:
predict_ae_from_upstream_model(data_test_2018_original, X_test, y_test)

338096/338096 [==============================] - 449s 1ms/step
              precision    recall  f1-score   support

           0       0.87      0.97      0.92   9493199
           1       0.03      0.01      0.01   1325845

    accuracy                           0.85  10819044
   macro avg       0.45      0.49      0.47  10819044
weighted avg       0.77      0.85      0.81  10819044

338096/338096 - 441s - loss: 0.1503 - acc: 0.8497 - f1_m: 0.0017 - precision_m: 0.0071 - recall_m: 0.0011 - 441s/epoch - 1ms/step


In [ ]:
del autoencoder

### Resampling

In [ ]:
X_test, y_test = get_x_y(data2018_original_resampling)

In [ ]:
del data2018_original_resampling

In [ ]:
data_test_2018_original_r = pd.concat([X_test, y_test], axis=1)

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [ ]:
predict_ae_from_upstream_model(data_test_2018_original_r, X_test, y_test, True)

82866/82866 [==============================] - 128s 1ms/step
              precision    recall  f1-score   support

           0       0.57      0.67      0.61   1325845
           1       0.60      0.49      0.54   1325845

    accuracy                           0.58   2651690
   macro avg       0.58      0.58      0.58   2651690
weighted avg       0.58      0.58      0.58   2651690

82866/82866 - 105s - loss: 0.4198 - acc: 0.5798 - f1_m: 0.2760 - precision_m: 0.3653 - recall_m: 0.2447 - 105s/epoch - 1ms/step


### 2018 improved

In [ ]:
X_test, y_test = get_x_y(data2018_corrected)
del data2018_corrected

data_test_2018_corrected = pd.concat([X_test, y_test], axis=1)

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [ ]:
predict_ae_from_upstream_model(data_test_2018_corrected, X_test, y_test)

305532/305532 [==============================] - 423s 1ms/step
              precision    recall  f1-score   support

           0       0.62      0.97      0.76   5935341
           1       0.62      0.07      0.12   3841659

    accuracy                           0.62   9777000
   macro avg       0.62      0.52      0.44   9777000
weighted avg       0.62      0.62      0.51   9777000

305532/305532 - 398s - loss: 0.3831 - acc: 0.6169 - f1_m: 0.0282 - precision_m: 0.0329 - recall_m: 0.0263 - 398s/epoch - 1ms/step


### resampling

In [ ]:
X_test, y_test = get_x_y(data2018_corrected_resampling)

In [ ]:
del data2018_corrected_resampling

In [ ]:
data_test_2018_corrected_r = pd.concat([X_test, y_test], axis=1)

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [ ]:
predict_ae_from_upstream_model(data_test_2018_corrected_r, X_test, y_test, True)

240104/240104 [==============================] - 323s 1ms/step
              precision    recall  f1-score   support

           0       0.65      0.81      0.72   3841654
           1       0.75      0.57      0.65   3841659

    accuracy                           0.69   7683313
   macro avg       0.70      0.69      0.69   7683313
weighted avg       0.70      0.69      0.69   7683313

240104/240104 - 311s - loss: 0.3110 - acc: 0.6897 - f1_m: 0.2927 - precision_m: 0.3431 - recall_m: 0.2861 - 311s/epoch - 1ms/step


### DNN

In [7]:
# define the model architecture
input_dim = 76 #num of feature columns

# Initialize the constructor, for dense layers added
dnn = tf.keras.Sequential()

# dnn.add(tf.keras.layers.Flatten()) #flattens the input data

# Add an input layer
dnn.add(tf.keras.layers.Dense(32, activation='sigmoid', input_dim=input_dim))

# Add one hidden layer
dnn.add(tf.keras.layers.Dense(8, activation='sigmoid'))

# Add an output layer
dnn.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
def predict_dnn_from_upstream_model(df, evaluate_x, evaluate_y, is_resampling = False):
  dnn.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['acc',f1_m,precision_m, recall_m])

  if (is_resampling):
    dnn.load_weights("/content/onedrive/models/dnn_classifier_resampling.ckpt")
  else:
    dnn.load_weights("/content/onedrive/models/dnn_classifier.ckpt")

  ypred = dnn.predict(evaluate_x)
  # Define a threshold for converting predictions to binary class labels
  threshold = 0.5

  # Convert predictions to binary class labels
  predicted_classes = (ypred >= threshold).astype(int)
  df['ypreds_dnn'] = predicted_classes
  print(classification_report(evaluate_y, predicted_classes))
  dnn.evaluate(evaluate_x, evaluate_y, verbose=2)

### 2017 improved

In [ ]:
predict_dnn_from_upstream_model(data_test_2017_corrected, X_test, y_test)

65625/65625 [==============================] - 71s 1ms/step
              precision    recall  f1-score   support

           0       0.76      0.99      0.86   1594540
           1       0.31      0.02      0.03    505431

    accuracy                           0.75   2099971
   macro avg       0.54      0.50      0.45   2099971
weighted avg       0.65      0.75      0.66   2099971

65625/65625 - 80s - loss: 1.4043 - acc: 0.7543 - f1_m: 0.0154 - precision_m: 0.0714 - recall_m: 0.0104 - 80s/epoch - 1ms/step


In [ ]:
data_test_2017_corrected.to_parquet('/content/onedrive/test_result/data_test_dl_tr_2017_corrected.parquet', index = False, compression=None, engine='fastparquet')

In [ ]:
pd.read_parquet('/content/onedrive/test_result/data_test_dl_tr_2017_corrected.parquet')

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label_encoded,ypreds_ae,ypreds_dnn
0,112740690,32,16,6448,1152,403,0,201.500000,204.724205,72,...,1.199802e+01,380,343,1.610540e+07,4.988048e+05,16399772,15375229,0,0,0
1,112740560,32,16,6448,5056,403,0,201.500000,204.724205,316,...,1.574499e+01,330,285,1.610543e+07,4.987937e+05,16399782,15375263,0,0,0
2,113757377,545,0,0,0,0,0,0.000000,0.000000,0,...,7.324646e+06,18851791,19,1.221036e+07,6.935824e+06,20757030,5504997,0,0,0
3,91997219,388,0,37151,0,227,37,95.750000,55.785320,0,...,1.152782e+07,24721964,16,1.319764e+07,5.826905e+06,19776791,5817470,0,0,0
4,66966070,6,6,288,288,48,48,48.000000,0.000000,48,...,0.000000e+00,1968172,1968172,6.497443e+07,0.000000e+00,64974431,64974431,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2099966,5074745,7,6,582,1204,582,0,83.142857,219.975323,602,...,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0
2099967,209,2,2,70,230,35,35,35.000000,0.000000,115,...,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0
2099968,116281383,24,21,699,5411,322,0,29.125000,79.231808,1448,...,8.804912e+04,328226,23012,9.639210e+06,1.321437e+06,10024910,5443180,0,0,0
2099969,149,2,2,72,104,36,36,36.000000,0.000000,52,...,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0


### Resampling

In [ ]:
predict_dnn_from_upstream_model(data_test_2017_corrected_r, X_test, y_test, True)

31590/31590 [==============================] - 36s 1ms/step
              precision    recall  f1-score   support

           0       0.44      0.38      0.41    505431
           1       0.45      0.52      0.49    505431

    accuracy                           0.45   1010862
   macro avg       0.45      0.45      0.45   1010862
weighted avg       0.45      0.45      0.45   1010862

31590/31590 - 38s - loss: 7.5412 - acc: 0.4485 - f1_m: 0.2858 - precision_m: 0.3817 - recall_m: 0.2602 - 38s/epoch - 1ms/step


In [ ]:
data_test_2017_corrected_r.to_parquet('/content/onedrive/test_result/data_test_dl_tr_2017_corrected_resampling.parquet', index = False, compression=None, engine='fastparquet')

In [ ]:
pd.read_parquet('/content/onedrive/test_result/data_test_dl_tr_2017_corrected_resampling.parquet')

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label_encoded,ypreds_ae,ypreds_dnn
0,115514613,17,16,450,282,439,0,26.470588,106.307407,282,...,1.992275e+04,84099,14933,9.598773e+06,1.386276e+06,10014002,5197043,0,0,0
1,195613,14,14,831,15880,357,0,59.357143,115.745481,2896,...,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0
2,117347,2,2,68,312,34,34,34.000000,0.000000,156,...,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0,0,0,1
3,5196055,9,11,1355,5738,712,0,150.555556,270.671901,1460,...,0.000000e+00,150774,150774,5.001138e+06,0.000000e+00,5001138,5001138,0,0,0
4,60744909,14,15,1158,4292,517,0,82.714286,186.590225,2097,...,1.029975e+05,277140,24618,1.005307e+07,3.207239e+05,10215374,9413421,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010857,656477,11,5,1041,11595,347,0,94.636364,162.083480,7240,...,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0,1,1,0
1010858,103680007,13,6,1472,11632,368,0,113.230769,176.781482,4344,...,4.668730e+05,829021,3207,3.426969e+07,5.022459e+07,92263198,5000791,1,1,0
1010859,11464350,9,5,588,11632,588,0,65.333333,196.000000,7288,...,0.000000e+00,749,749,6.423318e+06,0.000000e+00,6423318,6423318,1,1,0
1010860,177006,8,8,322,11595,322,0,40.250000,113.844192,7240,...,0.000000e+00,0,0,0.000000e+00,0.000000e+00,0,0,1,0,0


### 2018 original

In [ ]:
predict_dnn_from_upstream_model(data_test_2018_original, X_test, y_test)

NameError: ignored

In [ ]:
data_test_2018_original.to_parquet('/content/onedrive/test_result/data_test_dl_tr_2018_original.parquet', index = False, compression=None, engine='fastparquet')

In [ ]:
del data_test_2018_original, X_test, y_test

In [ ]:
pd.read_parquet('/content/onedrive/test_result/data_test_dl_tr_2018_original.parquet')

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label_encoded,ypreds_ae,ypreds_dnn
0,141385.0,9.0,7.0,553.0,3773.0,202.0,0.0,61.444444,87.534438,1460.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
1,281.0,2.0,1.0,38.0,0.0,38.0,0.0,19.000000,26.870058,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
2,279824.0,11.0,15.0,1086.0,10527.0,385.0,0.0,98.727273,129.392497,1460.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
3,132.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
4,274016.0,9.0,13.0,1285.0,6141.0,517.0,0.0,142.777778,183.887722,1460.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10819039,20.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1
10819040,105445.0,2.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
10819041,733880.0,2.0,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
10819042,732728.0,2.0,2.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1


### Resampling

In [ ]:
predict_dnn_from_upstream_model(data_test_2018_original_r, X_test, y_test, True)

82866/82866 [==============================] - 93s 1ms/step
              precision    recall  f1-score   support

           0       0.66      0.49      0.56   1325845
           1       0.59      0.74      0.66   1325845

    accuracy                           0.62   2651690
   macro avg       0.62      0.62      0.61   2651690
weighted avg       0.62      0.62      0.61   2651690

82866/82866 - 104s - loss: 3.4703 - acc: 0.6156 - f1_m: 0.3862 - precision_m: 0.4265 - recall_m: 0.3725 - 104s/epoch - 1ms/step


In [ ]:
data_test_2018_original_r.to_parquet('/content/onedrive/test_result/data_test_dl_tr_2018_original_resampling.parquet', index = False, compression=None, engine='fastparquet')

In [ ]:
del data_test_2018_original_r, X_test, y_test

In [ ]:
pd.read_parquet('/content/onedrive/test_result/data_test_dl_tr_2018_original_resampling.parquet')

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label_encoded,ypreds_ae,ypreds_dnn
0,60134050.0,9.0,7.0,2044.0,4174.0,837.0,0.0,227.111111,350.915746,1460.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0
1,1103521.0,8.0,7.0,1148.0,1581.0,677.0,0.0,143.500000,228.129662,1173.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2,10086791.0,7.0,9.0,1046.0,5283.0,731.0,0.0,149.428571,267.599114,1460.0,...,0.0,208480.0,208480.0,9878310.0,0.0,9878310.0,9878310.0,0,1,0
3,85929.0,2.0,2.0,74.0,262.0,37.0,37.0,37.000000,0.000000,131.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1
4,303073.0,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2651685,137728.0,2.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1
2651686,11172035.0,5.0,5.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0
2651687,22.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1
2651688,20.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1


### 2018 improved

In [ ]:
predict_dnn_from_upstream_model(data_test_2018_corrected, X_test, y_test)

305532/305532 [==============================] - 360s 1ms/step
              precision    recall  f1-score   support

           0       0.60      0.96      0.74   5935341
           1       0.02      0.00      0.00   3841659

    accuracy                           0.58   9777000
   macro avg       0.31      0.48      0.37   9777000
weighted avg       0.37      0.58      0.45   9777000

305532/305532 - 388s - loss: 2.5857 - acc: 0.5822 - f1_m: 0.0011 - precision_m: 0.0051 - recall_m: 6.3366e-04 - 388s/epoch - 1ms/step


In [ ]:
data_test_2018_corrected.to_parquet('/content/onedrive/test_result/data_test_dl_tr_2018_corrected.parquet', index = False, compression=None, engine='fastparquet')

In [ ]:
del data_test_2018_corrected, X_test, y_test

In [ ]:
pd.read_parquet('/content/onedrive/test_result/data_test_dl_tr_2018_corrected.parquet')

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label_encoded,ypreds_ae,ypreds_dnn
0,115985546,20,19,1914,1268,1304,0,95.700000,306.775093,1085,...,60803.836874,231615,20927,9.625176e+06,1.320240e+06,10025690,5433219,0,0,0
1,60837973,12,11,323,337,317,0,26.916667,91.353916,337,...,63152.965095,170207,11714,1.007032e+07,3.421857e+04,10088042,10000748,0,0,0
2,1227,1,1,42,102,42,42,42.000000,0.000000,102,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0
3,887,1,1,52,108,52,52,52.000000,0.000000,108,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0
4,1298,1,1,42,138,42,42,42.000000,0.000000,138,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9776995,2,1,1,0,0,0,0,0.000000,0.000000,0,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,1,0,0
9776996,2,1,1,0,0,0,0,0.000000,0.000000,0,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,1,0,0
9776997,2,1,1,0,0,0,0,0.000000,0.000000,0,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,1,0,0
9776998,2,1,1,0,0,0,0,0.000000,0.000000,0,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,1,0,0


### resampling

In [ ]:
predict_dnn_from_upstream_model(data_test_2018_corrected_r, X_test, y_test)


240104/240104 [==============================] - 279s 1ms/step
              precision    recall  f1-score   support

           0       0.49      0.95      0.65   3841654
           1       0.27      0.02      0.03   3841659

    accuracy                           0.49   7683313
   macro avg       0.38      0.49      0.34   7683313
weighted avg       0.38      0.49      0.34   7683313

240104/240104 - 302s - loss: 3.0581 - acc: 0.4856 - f1_m: 0.0131 - precision_m: 0.0477 - recall_m: 0.0084 - 302s/epoch - 1ms/step


In [ ]:
data_test_2018_corrected_r.to_parquet('/content/onedrive/test_result/data_test_dl_tr_2018_corrected_resampling.parquet', index = False, compression=None, engine='fastparquet')

In [ ]:
del data_test_2018_corrected_r

In [ ]:
pd.read_parquet('/content/onedrive/test_result/data_test_dl_tr_2018_corrected_resampling.parquet')

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label_encoded,ypreds_ae,ypreds_dnn
0,5298120,11,10,1815,1933,677,0,165.000000,261.498375,1173,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0
1,90103210,13,11,1442,1731,725,0,110.923077,196.522035,1179,...,0.000000,4060528,4060528,8.604268e+07,0.000000e+00,86042682,86042682,0,0,0
2,40164,4,2,77,0,46,0,19.250000,23.056091,0,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,0,1,1
3,1722788,8,7,1128,1581,661,0,141.000000,222.623321,1173,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0
4,23258044,9,10,1148,2754,677,0,127.555556,218.691056,1173,...,321757.539778,670442,34380,7.370407e+06,2.640889e+06,10329404,5252514,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7683308,2,1,1,0,0,0,0,0.000000,0.000000,0,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,1,0,0
7683309,2,1,1,0,0,0,0,0.000000,0.000000,0,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,1,0,0
7683310,2,1,1,0,0,0,0,0.000000,0.000000,0,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,1,0,0
7683311,2,1,1,0,0,0,0,0.000000,0.000000,0,...,0.000000,0,0,0.000000e+00,0.000000e+00,0,0,1,0,0


### Retrain the model, fine tuning

In [8]:
# data_test_2017_corrected = pd.read_parquet('/content/onedrive/test_result/data_test_dl_tr_2017_corrected.parquet', engine="fastparquet")
# data_test_2017_corrected_r = pd.read_parquet('/content/onedrive/test_result/data_test_dl_tr_2017_corrected_resampling.parquet', engine="fastparquet")
# data_test_2018_original = pd.read_parquet('/content/onedrive/test_result/data_test_dl_tr_2018_original.parquet', engine="fastparquet")
# data_test_2018_original_r = pd.read_parquet('/content/onedrive/test_result/data_test_dl_tr_2018_original_resampling.parquet', engine="fastparquet")
data_test_2018_corrected = pd.read_parquet('/content/onedrive/test_result/data_test_dl_tr_2018_corrected.parquet', engine="fastparquet")
# data_test_2018_corrected_r = pd.read_parquet('/content/onedrive/test_result/data_test_dl_tr_2018_corrected_resampling.parquet', engine="fastparquet")

In [9]:
def gen_misclassified_data(df, y):
  return df[df['label_encoded'] != df[y]]

In [10]:
def gen_train_x_y(df, is_resampling = False):
  X_retrained = df.loc[:, 'Flow Duration': 'Idle Min']
  y_retrained = df['label_encoded']

  scaler = StandardScaler()
  X_retrained = scaler.fit_transform(X_retrained)

  return X_retrained, y_retrained

In [11]:
epochs = 16
batch_size = 128

def retrain_autoencoder(df, X_test, y_test, is_resampling):


  misclassfied_data = gen_misclassified_data(df, 'ypreds_ae')
  X_train, y_train = gen_train_x_y(misclassfied_data, is_resampling)


  model = "/content/onedrive/models/autoencoder_classifier.ckpt"

  if is_resampling:
    model = "/content/onedrive/models/autoencoder_classifier_resampling.ckpt"

  autoencoder = AutoEncoder()
  autoencoder.load_weights(model)


  autoencoder.compile(loss='mae',
                    optimizer='rmsprop',
                    metrics=['acc',f1_m, precision_m, recall_m])

  autoencoder.fit(X_train, y_train,
                  epochs=epochs,
                  batch_size=batch_size,
                  shuffle=True,
                  validation_split=0.1,
                  verbose=1)

  ypred = autoencoder.predict(X_test)
  # Define a threshold for converting predictions to binary c
  threshold = 0.5
  # Convert predictions to binary class labels
  predicted_classes = (ypred >= threshold).astype(int)
  print(classification_report(y_test, predicted_classes))

In [12]:
batch_size = 32
def retrain_dnn(df, X_test, y_test, is_resampling):

  misclassfied_data = gen_misclassified_data(df, 'ypreds_dnn')
  X_train, y_train = gen_train_x_y(misclassfied_data, is_resampling)


  model = "/content/onedrive/models/dnn_classifier.ckpt"

  if is_resampling:
    model = "/content/onedrive/models/dnn_classifier_resampling.ckpt"

  dnn.load_weights(model)


  dnn.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['acc',f1_m,precision_m, recall_m])

  dnn.fit(X_train, y_train,
                      epochs=epochs,
                      batch_size=batch_size,
                      shuffle=True,
                      validation_split=0.1,
                      verbose=1)

  ypred = dnn.predict(X_test)
  # Define a threshold for converting predictions to binary class
  threshold = 0.5
  # Convert predictions to binary class labels
  predicted_classes = (ypred >= threshold).astype(int)
  print(classification_report(y_test, predicted_classes))


### 2017 improved

In [17]:
X_test, y_test = get_x_y(data2017_corrected)

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [18]:
del data2017_corrected

In [19]:
retrain_autoencoder(data_test_2017_corrected, X_test, y_test, False)

Epoch 1/16
2108/2108 [==============================] - 101s 4ms/step - loss: 0.9554 - acc: 0.0446 - f1_m: 1.5247e-05 - precision_m: 5.9298e-04 - recall_m: 7.7773e-06 - val_loss: 0.9069 - val_acc: 0.0931 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/16
2108/2108 [==============================] - 8s 4ms/step - loss: 0.9551 - acc: 0.0449 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 0.9069 - val_acc: 0.0931 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 3/16
2108/2108 [==============================] - 8s 4ms/step - loss: 0.9551 - acc: 0.0449 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - val_loss: 0.9069 - val_acc: 0.0931 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 4/16
2108/2108 [==============================] - 9s 4ms/step - loss: 0.9551 - acc: 0.0449 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.76      1.00      0.86   1594540
           1       0.00      0.00      0.00    505431

    accuracy                           0.76   2099971
   macro avg       0.38      0.50      0.43   2099971
weighted avg       0.58      0.76      0.66   2099971



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
retrain_dnn(data_test_2017_corrected, X_test, y_test, False)

Epoch 1/16
3628/3628 [==============================] - 14s 3ms/step - loss: 0.7039 - acc: 0.7666 - f1_m: 0.8171 - precision_m: 0.9745 - recall_m: 0.7726 - val_loss: 0.0689 - val_acc: 0.9581 - val_f1_m: 0.9675 - val_precision_m: 0.9582 - val_recall_m: 0.9851
Epoch 2/16
3628/3628 [==============================] - 13s 3ms/step - loss: 0.0285 - acc: 0.9855 - f1_m: 0.9925 - precision_m: 0.9931 - recall_m: 0.9921 - val_loss: 0.0630 - val_acc: 0.9669 - val_f1_m: 0.9652 - val_precision_m: 0.9843 - val_recall_m: 0.9490
Epoch 3/16
3628/3628 [==============================] - 13s 4ms/step - loss: 0.0221 - acc: 0.9916 - f1_m: 0.9956 - precision_m: 0.9999 - recall_m: 0.9914 - val_loss: 0.0551 - val_acc: 0.9725 - val_f1_m: 0.9690 - val_precision_m: 0.9844 - val_recall_m: 0.9555
Epoch 4/16
3628/3628 [==============================] - 13s 3ms/step - loss: 0.0176 - acc: 0.9941 - f1_m: 0.9969 - precision_m: 0.9999 - recall_m: 0.9940 - val_loss: 0.0488 - val_acc: 0.9786 - val_f1_m: 0.9725 - val_precisi

In [23]:
del data_test_2017_corrected, X_test, y_test

### resampling

In [ ]:
X_test, y_test = get_x_y(data2017_corrected_resampling)

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [ ]:
del data2017_corrected_resampling

In [ ]:
retrain_autoencoder(data_test_2017_corrected_r, X_test, y_test, True)

Epoch 1/16
1346/1346 [==============================] - 36s 3ms/step - loss: 0.6630 - acc: 0.9203 - f1_m: 0.9359 - precision_m: 0.9640 - recall_m: 0.9107 - val_loss: 0.0195 - val_acc: 0.9931 - val_f1_m: 0.9965 - val_precision_m: 1.0000 - val_recall_m: 0.9930
Epoch 2/16
1346/1346 [==============================] - 3s 2ms/step - loss: 0.1761 - acc: 0.9541 - f1_m: 0.9645 - precision_m: 0.9951 - recall_m: 0.9362 - val_loss: 0.0063 - val_acc: 0.9968 - val_f1_m: 0.9984 - val_precision_m: 1.0000 - val_recall_m: 0.9968
Epoch 3/16
1346/1346 [==============================] - 3s 2ms/step - loss: 0.0850 - acc: 0.9524 - f1_m: 0.9633 - precision_m: 0.9912 - recall_m: 0.9373 - val_loss: 0.0075 - val_acc: 0.9960 - val_f1_m: 0.9980 - val_precision_m: 1.0000 - val_recall_m: 0.9960
Epoch 4/16
1346/1346 [==============================] - 3s 2ms/step - loss: 0.0534 - acc: 0.9772 - f1_m: 0.9827 - precision_m: 0.9856 - recall_m: 0.9803 - val_loss: 0.0082 - val_acc: 0.9970 - val_f1_m: 0.9985 - val_precision_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.50      1.00      0.67    505431
           1       0.00      0.00      0.00    505431

    accuracy                           0.50   1010862
   macro avg       0.25      0.50      0.33   1010862
weighted avg       0.25      0.50      0.33   1010862



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
retrain_dnn(data_test_2017_corrected_r, X_test, y_test, True)


Epoch 1/16
3920/3920 [==============================] - 77s 3ms/step - loss: 0.0190 - acc: 0.9810 - f1_m: 0.9741 - precision_m: 0.9905 - recall_m: 0.9591 - val_loss: 0.0062 - val_acc: 0.9938 - val_f1_m: 0.9968 - val_precision_m: 1.0000 - val_recall_m: 0.9937
Epoch 2/16
3920/3920 [==============================] - 10s 3ms/step - loss: 0.0109 - acc: 0.9891 - f1_m: 0.9850 - precision_m: 0.9987 - recall_m: 0.9719 - val_loss: 0.0043 - val_acc: 0.9957 - val_f1_m: 0.9978 - val_precision_m: 1.0000 - val_recall_m: 0.9957
Epoch 3/16
3920/3920 [==============================] - 11s 3ms/step - loss: 0.0108 - acc: 0.9892 - f1_m: 0.9852 - precision_m: 0.9990 - recall_m: 0.9720 - val_loss: 0.0051 - val_acc: 0.9949 - val_f1_m: 0.9974 - val_precision_m: 1.0000 - val_recall_m: 0.9948
Epoch 4/16
3920/3920 [==============================] - 11s 3ms/step - loss: 0.0107 - acc: 0.9893 - f1_m: 0.9853 - precision_m: 0.9993 - recall_m: 0.9719 - val_loss: 0.0055 - val_acc: 0.9945 - val_f1_m: 0.9972 - val_precisi

In [ ]:
del data_test_2017_corrected_r, X_test, y_test


### 2018 original

In [30]:
X_test, y_test = get_x_y(data2018_original)
del data2018_original

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [31]:
retrain_autoencoder(data_test_2018_original, X_test, y_test, False)

Epoch 1/16
11432/11432 [==============================] - 136s 4ms/step - loss: 0.7498 - acc: 0.2502 - f1_m: 0.1393 - precision_m: 0.9988 - recall_m: 0.0755 - val_loss: 0.3487 - val_acc: 0.6513 - val_f1_m: 0.4610 - val_precision_m: 0.4894 - val_recall_m: 0.4469
Epoch 2/16
11432/11432 [==============================] - 46s 4ms/step - loss: 0.7496 - acc: 0.2504 - f1_m: 0.1396 - precision_m: 0.9994 - recall_m: 0.0756 - val_loss: 0.3487 - val_acc: 0.6513 - val_f1_m: 0.4610 - val_precision_m: 0.4894 - val_recall_m: 0.4469
Epoch 3/16
11432/11432 [==============================] - 46s 4ms/step - loss: 0.7496 - acc: 0.2504 - f1_m: 0.1395 - precision_m: 0.9997 - recall_m: 0.0756 - val_loss: 0.3487 - val_acc: 0.6513 - val_f1_m: 0.4610 - val_precision_m: 0.4894 - val_recall_m: 0.4469
Epoch 4/16
11432/11432 [==============================] - 46s 4ms/step - loss: 0.7496 - acc: 0.2504 - f1_m: 0.1396 - precision_m: 0.9995 - recall_m: 0.0756 - val_loss: 0.3487 - val_acc: 0.6513 - val_f1_m: 0.4610 - va

In [32]:
retrain_dnn(data_test_2018_original, X_test, y_test, False)

Epoch 1/16
21309/21309 [==============================] - 114s 4ms/step - loss: 0.5352 - acc: 0.7648 - f1_m: 0.4635 - precision_m: 0.9923 - recall_m: 0.3066 - val_loss: 1.0488 - val_acc: 0.1962 - val_f1_m: 0.0076 - val_precision_m: 0.1280 - val_recall_m: 0.0040
Epoch 2/16
21309/21309 [==============================] - 74s 3ms/step - loss: 0.5042 - acc: 0.7738 - f1_m: 0.4912 - precision_m: 0.9991 - recall_m: 0.3299 - val_loss: 0.3166 - val_acc: 0.8714 - val_f1_m: 0.6979 - val_precision_m: 0.8202 - val_recall_m: 0.6830
Epoch 3/16
21309/21309 [==============================] - 75s 3ms/step - loss: 0.2175 - acc: 0.9256 - f1_m: 0.8631 - precision_m: 0.9988 - recall_m: 0.7803 - val_loss: 0.2811 - val_acc: 0.9284 - val_f1_m: 0.7736 - val_precision_m: 0.8428 - val_recall_m: 0.7521
Epoch 4/16
21309/21309 [==============================] - 76s 4ms/step - loss: 0.0154 - acc: 0.9976 - f1_m: 0.9964 - precision_m: 0.9995 - recall_m: 0.9935 - val_loss: 0.0140 - val_acc: 0.9973 - val_f1_m: 0.8511 - va

In [33]:
del data_test_2018_original, X_test, y_test

### resampling

In [ ]:
X_test, y_test = get_x_y(data2018_original_resampling)

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [ ]:
del data2018_original_resampling


In [ ]:
retrain_autoencoder(data_test_2018_original_r, X_test, y_test, True)


Epoch 1/16
7835/7835 [==============================] - 48s 2ms/step - loss: 0.1237 - acc: 0.9635 - f1_m: 0.9659 - precision_m: 0.9864 - recall_m: 0.9489 - val_loss: 1.3003 - val_acc: 0.5009 - val_f1_m: 0.5423 - val_precision_m: 0.9575 - val_recall_m: 0.5011
Epoch 2/16
7835/7835 [==============================] - 17s 2ms/step - loss: 0.0114 - acc: 0.9970 - f1_m: 0.9973 - precision_m: 0.9985 - recall_m: 0.9962 - val_loss: 1.5727 - val_acc: 0.7889 - val_f1_m: 0.8278 - val_precision_m: 0.9977 - val_recall_m: 0.7890
Epoch 3/16
7835/7835 [==============================] - 18s 2ms/step - loss: 0.0072 - acc: 0.9982 - f1_m: 0.9984 - precision_m: 0.9993 - recall_m: 0.9975 - val_loss: 1.6148 - val_acc: 0.7955 - val_f1_m: 0.8316 - val_precision_m: 0.9977 - val_recall_m: 0.7956
Epoch 4/16
7835/7835 [==============================] - 19s 2ms/step - loss: 0.0056 - acc: 0.9985 - f1_m: 0.9987 - precision_m: 0.9993 - recall_m: 0.9980 - val_loss: 1.5024 - val_acc: 0.8007 - val_f1_m: 0.8344 - val_precisi

In [ ]:
retrain_dnn(data_test_2018_original_r, X_test, y_test, True)


Epoch 1/16
7167/7167 [==============================] - 85s 3ms/step - loss: 0.2251 - acc: 0.7749 - f1_m: 0.3127 - precision_m: 0.7761 - recall_m: 0.2028 - val_loss: 0.7584 - val_acc: 0.2417 - val_f1_m: 0.2620 - val_precision_m: 0.3488 - val_recall_m: 0.2415
Epoch 2/16
7167/7167 [==============================] - 21s 3ms/step - loss: 0.1969 - acc: 0.8031 - f1_m: 0.4005 - precision_m: 0.9107 - recall_m: 0.2648 - val_loss: 0.1153 - val_acc: 0.8847 - val_f1_m: 0.9052 - val_precision_m: 0.9849 - val_recall_m: 0.8841
Epoch 3/16
7167/7167 [==============================] - 20s 3ms/step - loss: 0.1649 - acc: 0.8351 - f1_m: 0.5253 - precision_m: 0.9774 - recall_m: 0.3688 - val_loss: 0.0592 - val_acc: 0.9409 - val_f1_m: 0.9582 - val_precision_m: 0.9849 - val_recall_m: 0.9403
Epoch 4/16
7167/7167 [==============================] - 21s 3ms/step - loss: 0.0127 - acc: 0.9873 - f1_m: 0.9725 - precision_m: 0.9933 - recall_m: 0.9571 - val_loss: 0.0497 - val_acc: 0.9504 - val_f1_m: 0.9642 - val_precisi

In [ ]:
del data_test_2018_original_r, X_test, y_test


### 2018 Improved

In [14]:
X_test, y_test = get_x_y(data2018_corrected)
del data2018_corrected

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [42]:
retrain_autoencoder(data_test_2018_corrected, X_test, y_test, False)

Epoch 1/16
26335/26335 [==============================] - 197s 4ms/step - loss: 0.9406 - acc: 0.0594 - f1_m: 0.0342 - precision_m: 0.8788 - recall_m: 0.0176 - val_loss: 0.9494 - val_acc: 0.0506 - val_f1_m: 0.0095 - val_precision_m: 0.2709 - val_recall_m: 0.0049
Epoch 2/16
26335/26335 [==============================] - 107s 4ms/step - loss: 0.8245 - acc: 0.1755 - f1_m: 0.2231 - precision_m: 0.9510 - recall_m: 0.1387 - val_loss: 0.1321 - val_acc: 0.8679 - val_f1_m: 0.8233 - val_precision_m: 0.8705 - val_recall_m: 0.8223
Epoch 3/16
26335/26335 [==============================] - 107s 4ms/step - loss: 0.7092 - acc: 0.2908 - f1_m: 0.4101 - precision_m: 0.9999 - recall_m: 0.2592 - val_loss: 0.1320 - val_acc: 0.8680 - val_f1_m: 0.8233 - val_precision_m: 0.8705 - val_recall_m: 0.8223
Epoch 4/16
26335/26335 [==============================] - 107s 4ms/step - loss: 0.7090 - acc: 0.2910 - f1_m: 0.4104 - precision_m: 0.9998 - recall_m: 0.2594 - val_loss: 0.1302 - val_acc: 0.8698 - val_f1_m: 0.8269 -

In [15]:
retrain_dnn(data_test_2018_corrected, X_test, y_test, False)

Epoch 1/16
28725/28725 [==============================] - 142s 3ms/step - loss: 0.2073 - acc: 0.9294 - f1_m: 0.9589 - precision_m: 0.9387 - recall_m: 0.9892 - val_loss: 0.2761 - val_acc: 0.9370 - val_f1_m: 0.9371 - val_precision_m: 0.9370 - val_recall_m: 0.9373
Epoch 2/16
28725/28725 [==============================] - 99s 3ms/step - loss: 0.0872 - acc: 0.9449 - f1_m: 0.9711 - precision_m: 0.9567 - recall_m: 0.9875 - val_loss: 0.0401 - val_acc: 0.9921 - val_f1_m: 0.9329 - val_precision_m: 0.9373 - val_recall_m: 0.9294
Epoch 3/16
28725/28725 [==============================] - 99s 3ms/step - loss: 0.0101 - acc: 0.9950 - f1_m: 0.9973 - precision_m: 1.0000 - recall_m: 0.9948 - val_loss: 0.0016 - val_acc: 0.9997 - val_f1_m: 0.9372 - val_precision_m: 0.9373 - val_recall_m: 0.9371
Epoch 4/16
28725/28725 [==============================] - 98s 3ms/step - loss: 3.8486e-04 - acc: 0.9999 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 0.0014 - val_acc: 0.9997 - val_f1_m: 0.9372 

In [16]:
del data_test_2018_corrected, X_test, y_test

### resampling

In [ ]:
X_test, y_test = get_x_y(data2018_corrected_resampling)

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

In [ ]:
del data2018_corrected_resampling


In [ ]:
retrain_autoencoder(data_test_2018_corrected_r, X_test, y_test, True)


In [29]:
retrain_dnn(data_test_2018_corrected_r, X_test, y_test, True)


Epoch 1/16
27788/27788 [==============================] - 139s 3ms/step - loss: 0.1016 - acc: 0.8984 - f1_m: 0.9414 - precision_m: 0.9984 - recall_m: 0.8944 - val_loss: 0.0118 - val_acc: 0.9882 - val_f1_m: 0.9879 - val_precision_m: 0.9951 - val_recall_m: 0.9832
Epoch 2/16
27788/27788 [==============================] - 73s 3ms/step - loss: 0.0677 - acc: 0.9323 - f1_m: 0.9629 - precision_m: 0.9999 - recall_m: 0.9290 - val_loss: 0.0118 - val_acc: 0.9882 - val_f1_m: 0.9879 - val_precision_m: 0.9951 - val_recall_m: 0.9832
Epoch 3/16
27788/27788 [==============================] - 69s 2ms/step - loss: 0.0676 - acc: 0.9324 - f1_m: 0.9630 - precision_m: 0.9999 - recall_m: 0.9290 - val_loss: 0.0118 - val_acc: 0.9882 - val_f1_m: 0.9879 - val_precision_m: 0.9951 - val_recall_m: 0.9832
Epoch 4/16
27788/27788 [==============================] - 69s 2ms/step - loss: 0.0676 - acc: 0.9324 - f1_m: 0.9630 - precision_m: 1.0000 - recall_m: 0.9290 - val_loss: 0.0118 - val_acc: 0.9882 - val_f1_m: 0.9879 - va

In [30]:
del data_test_2018_corrected_r, X_test, y_test
